# **Preproccsing the Sound**::

In [14]:
# Step 1: Install necessary libraries
# !pip install librosa matplotlib tensorflow

# Step 2: Import necessary libraries
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from PIL import Image

# Step 3: Function to convert audio to spectrogram and save it as an image
def convert_audio_to_spectrogram(audio_path, output_dir):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        mel_spect = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spect_db = librosa.power_to_db(mel_spect, ref=np.max)

        # Plot and save the spectrogram
        plt.figure(figsize=(4, 4))
        librosa.display.specshow(mel_spect_db, sr=sr, x_axis='time', y_axis='mel')
        plt.axis('off')  # No axis for the image
        output_image_path = os.path.join(output_dir, os.path.basename(audio_path).replace(".wav", ".png"))
        plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")

# Step 4: Process the heart sounds dataset and convert them to images
def process_heart_sounds(dataset_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(dataset_dir):
        if file_name.endswith('.wav'):
            audio_path = os.path.join(dataset_dir, file_name)
            convert_audio_to_spectrogram(audio_path, output_dir)

# Example paths for training and validation datasets
train_dataset_dir = '/content/phy_training/training-a/'  # Replace with your training dataset folder path
val_dataset_dir = '/content/phy_validation/validation/'  # Replace with your validation dataset folder path

# Example output paths for images
train_output_dir = '/content/heart_sounds_images/train/'
val_output_dir = '/content/heart_sounds_images/val/'

# Process and convert heart sounds to spectrogram images
process_heart_sounds(train_dataset_dir, train_output_dir)
process_heart_sounds(val_dataset_dir, val_output_dir)

# Step 5: Load spectrogram images into a TensorFlow image data generator
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    '/content/heart_sounds_images/train/',
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if you have more than 2 classes
)

val_generator = val_datagen.flow_from_directory(
    '/content/heart_sounds_images/val/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if needed
)

# Step 6: Build a simple CNN model for image classification
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # For binary classification

model.summary()

# Step 7: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Step 9: Evaluate the model on validation set
val_loss, val_acc = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f"Validation Accuracy: {val_acc:.2f}")

# Step 10: Visualize training history
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 12, 12, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,601,153 (9.92 MB)

 Trainable params: 2,601,153 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

ValueError: Must provide at least one structure